In [32]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

### Предобработка данных

In [33]:
def time_filter(data, days=2):
    
    """Фильтрация данных до порогового значения"""
    
    # создаем таблицу с первым и последним действием юзера
    min_max_user_time = data.groupby('user_id').agg({'timestamp': 'min'}) \
                            .rename(columns={'timestamp': 'min_timestamp'}) \
                            .reset_index()
    
    data_time_filtered = pd.merge(data, min_max_user_time, on='user_id', how='outer')
    
    # отбираем те записи, которые не позднее двух дней с начала учебы
    learning_time_threshold = days * 24 * 60 * 60
    data_time_filtered = data_time_filtered.query("timestamp <= min_timestamp + @learning_time_threshold")
    
    assert data_time_filtered.user_id.nunique() == data.user_id.nunique()
    
    return data_time_filtered.drop(['min_timestamp'], axis=1)

In [34]:
def base_features(events_data, submission_data):
    
    """Создание датасета с базовыми фичами: действия юзера 
    и правильные\неправильные ответы"""
    
    # построим таблицу со всеми действиями юзеров
    users_events_data = pd.pivot_table(data=events_data, values='step_id',
                                   index='user_id', columns='action',
                                   aggfunc='count', fill_value=0) \
                                   .reset_index() \
                                   .rename_axis('', axis=1)
    
    # таблица с колво правильных и неправильных попыток
    users_scores = pd.pivot_table(data=submission_data, 
                              values='step_id',
                              index='user_id',
                              columns='submission_status',
                              aggfunc='count',
                              fill_value=0).reset_index() \
                              .rename_axis('', axis=1)
    
    # соединяем в один датасет
    users_data = pd.merge(users_scores, users_events_data, on='user_id', how='outer').fillna(0)
    
    assert users_data.user_id.nunique() == events_data.user_id.nunique()
    
    return users_data

In [35]:
def target(submission_data, threshold=40):
    
    """Вычисление целевой переменной. Если юзер сделал 40 практический заданий,
    то будем считать, что он пройдет курс до конца"""
    
    # считаем колво решенных заданий у каждого пользователя
    users_count_correct = submission_data[submission_data.submission_status == 'correct'] \
                .groupby('user_id').agg({'step_id': 'count'}) \
                .reset_index().rename(columns={'step_id': 'corrects'})
    
    # если юзер выполнил нужное колво заданий, то он пройдет курс до конца
    users_count_correct['passed_course'] = (users_count_correct.corrects >= threshold).astype('int')
    
    return users_count_correct.drop(['corrects'], axis=1)

In [36]:
def time_features(events_data):
    
    """Создание временных фичей"""
    
    # добавление колонок с датами
    events_data['date'] = pd.to_datetime(events_data['timestamp'], unit='s')
    events_data['day'] = events_data['date'].dt.date
    
    # создаем таблицу с первым\последним действием юзера и колвом уникальных дней, проведенных на курсе
    users_time_feature = events_data.groupby('user_id').agg({'timestamp': ['min', 'max'], 'day': 'nunique'}) \
                        .droplevel(level=0, axis=1) \
                        .rename(columns={'nunique': 'days'}) \
                        .reset_index()
    
   
    
    return users_time_feature.drop(['max', 'min'], axis=1)

In [37]:
def steps_tried(submission_data):
    
    """Создание фичи с колвом уникальных шагов, которые пользователь пытался выполнить"""
    
    # сколько степов юзер попытался сделать
    steps_tried = submission_data.groupby('user_id').agg({'step_id' : 'nunique' }).reset_index() \
                                        .rename(columns={'step_id': 'steps_tried'})
    
    return steps_tried

In [38]:
def correct_ratio(data):
    
    """Создание фичи с долей правильных ответов"""
    
    data['correct_ratio'] = (data.correct / (data.correct + data.wrong)).fillna(0)
    
    return data

### Создание датасетов

In [39]:
def create_df(events_data, submission_data):
    
    """функция для формирования X датасета и y с целевыми переменными"""
    
    # фильтруем данные по дням от начала учебы
    events_2days = time_filter(events_data)
    submissions_2days = time_filter(submission_data)
    
    # создаем таблицу с базовыми фичами
    users_data = base_features(events_2days, submissions_2days)
    
    # создаем целевую переменную
    users_target_feature = target(submission_data, threshold=40)
    
    # создаем таблицу с временными фичами
    users_time_feature = time_features(events_2days)
    
    # создаем фичи с попытками степов и долей правильных ответов
    users_steps_tried = steps_tried(submissions_2days)
    users_data = correct_ratio(users_data)
    
    # соединяем шаги
    first_merge = users_data.merge(users_steps_tried, how='outer').fillna(0)
    
    # соединяем фичи со временем
    second_merge = first_merge.merge(users_time_feature, how='outer')
    
    # присоединяем целевую переменную
    third_merge = second_merge.merge(users_target_feature, how='outer').fillna(0)
    
    # отделяем целевую переменную и удаляем ее из основного датасета
    y = third_merge['passed_course'].map(int)
    X = third_merge.drop(['passed_course'], axis=1)
    
    return X, y

In [40]:
def create_test_df(events_data, submission_data):
    
    """функция для формирования test датасета без целевой переменной"""
    
    # фильтруем данные по дням от начала учебы
    events_2days = time_filter(events_data)
    submissions_2days = time_filter(submission_data)
    
    # создаем таблицу с базовыми фичами
    users_data = base_features(events_2days, submissions_2days)
    
    
    # создаем таблицу с временными фичами
    users_time_feature = time_features(events_2days)
    
    # создаем фичи с попытками степов и долей правильных ответов
    users_steps_tried = steps_tried(submissions_2days)
    users_data = correct_ratio(users_data)
    
    # соединяем шаги
    first_merge = users_data.merge(users_steps_tried, how='outer').fillna(0)
    
    # соединяем фичи со временем
    X = first_merge.merge(users_time_feature, how='outer')
       
    return X

### Загрузка данных

In [41]:
# загрузка тренировочного датасета
events_data_train = pd.read_csv('D:\Загрузки/event_data_train.csv')
submission_data_train = pd.read_csv('D:\Загрузки/submissions_data_train.csv')

In [42]:
# создание тренировочного датасета с нужными фичами и целевой переменной
X_train, y = create_df(events_data_train, submission_data_train)

In [43]:
# загрузка тестового датасета
events_data_test = pd.read_csv('D:\Загрузки/events_data_test.csv')
submission_data_test = pd.read_csv('D:\Загрузки/submission_data_test.csv')

In [44]:
# создание тестового датасета
X_test = create_test_df(events_data_test, submission_data_test)

### Обучение модели

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [46]:
def random_final_improved(train_data, y, test_data, size=0.20):
    
    """Улучшенный пайплайн с предобработкой данных"""
    
    test_data = test_data.sort_values('user_id')
    
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, y, test_size=size, random_state=42)

    numeric_features = [col for col in X_train.columns if col != 'user_id']
    
    # Создаем препроцессор
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features)  # Масштабируем только числовые фичи
        ]
        # remainder по умолчанию 'drop' - остальные колонки (user_id) удаляются
    )
    
    # Создаем комплексный пайплайн
    pipe = make_pipeline(
        preprocessor, # Масштабирование признаков
        RandomForestClassifier(
            max_depth=6, 
            n_estimators=28,  
            random_state=42,
            min_samples_split=12
        )
    )
  
    # Обучаем пайплайн
    pipe.fit(X_train, y_train)
    
    # Предсказания на валидации
    ypred_prob = pipe.predict_proba(X_valid)
    
    roc_score = roc_auc_score(y_valid, ypred_prob[:, 1])
    score = pipe.score(X_valid, y_valid)
    print(f"Правильность на валид наборе: {score:.3f}")
    print(f"Roc_auc_score на валид наборе: {roc_score:.5f}")
    
    # Предсказания на тестовых данных
    ypred_prob_final = pipe.predict_proba(test_data)
    result = test_data['user_id'].to_frame()
    result['is_gone'] = ypred_prob_final[:, 1]
    result[['user_id', 'is_gone']].to_csv(f'my_predict_improved_{roc_score:.5f}.csv', index=False)
    print(f'Результаты записаны в файл my_predict_improved_{roc_score:.5f}.csv')

    return pipe

In [47]:
# Дополнительная функция для подбора параметров пайплайна
def random_with_grid_improved(train_data, y, size=0.20):
    
    """Поиск наилучших параметров для улучшенного пайплайна"""
    
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, y, test_size=size, random_state=42)
    
    # Параметры для GridSearch
    param_grid = {
        'randomforestclassifier__n_estimators': range(25,33),
        'randomforestclassifier__max_depth': range(5,8),
        'randomforestclassifier__min_samples_split': range(10,15)
    }
    
    # Создаем пайплайн
    pipe = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler(),
        SelectKBest(f_classif),
        RandomForestClassifier(random_state=42, class_weight='balanced')
    )
    
    # Поиск по сетке параметров
    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1, scoring='roc_auc')
    grid.fit(X_train, y_train)
    
    print(f"Наилучшие параметры: {grid.best_params_}")
    print(f"Лучший ROC-AUC на кросс-валидации: {grid.best_score_:.5f}")
    
    # Оценка на валидации
    ypred_prob = grid.predict_proba(X_valid)
    roc_score = roc_auc_score(y_valid, ypred_prob[:, 1])
    print(f"ROC-AUC на валидации: {roc_score:.5f}")
    
    return grid

In [48]:
# Запуск улучшенной версии
print("=== УЛУЧШЕННЫЙ ПАЙПЛАЙН ===")
improved_model = random_final_improved(X_train, y, X_test)

=== УЛУЧШЕННЫЙ ПАЙПЛАЙН ===
Правильность на валид наборе: 0.904
Roc_auc_score на валид наборе: 0.87759
Результаты записаны в файл my_predict_improved_0.87759.csv


In [49]:
#Опционально: поиск лучших параметров (может занять время)
#print("=== ПОДБОР ПАРАМЕТРОВ ===")
#best_grid = random_with_grid_improved(X_train, y)